In [1]:
import pandas as pd
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt

try:
    df = pd.read_csv('OnlineRetail.csv', encoding = 'ISO-8859-1')
except:
    df = pd.read_csv('OnlineRetail.csv')
#for printing first 5 rows of dataframe
display(df.head())

#clearing data
print(f"size before cleaning: {df.shape}")

#delete rows with missing CustomerID
df = df.dropna(subset=['CustomerID'])
#removing duplicates
df = df.drop_duplicates()

#clear refunds and free
df = df[(df['Quantity'] < 0) & (df['UnitPrice'] > 0)]

print(f"size after cleaning: {df.shape}")

#sum checks
df['TotalSum'] = df['Quantity'] * df['UnitPrice']




,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


size before cleaning: (541909, 8)
size after cleaning: (8872, 8)


In [ ]:
#zzz